# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,28.44,74,40,6.69,US,2024-09-15
1,1,nuuk,64.1835,-51.7216,3.34,100,100,1.03,GL,2024-09-15
2,2,port-aux-francais,-49.3500,70.2167,2.39,74,84,13.63,TF,2024-09-15
3,3,grytviken,-54.2811,-36.5092,-2.59,75,63,4.25,GS,2024-09-15
4,4,myanaung,18.2833,95.3167,23.38,95,100,1.45,MM,2024-09-15


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[((city_data_df["Max Temp"] <= 27.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] <= 4.5),:]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how="any")

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bethel,41.3712,-73.4140,24.77,65,0,3.09,US,2024-09-15
117,117,tostado,-29.2320,-61.7692,20.26,28,0,3.76,AR,2024-09-15
141,141,mutuali,-14.8706,37.0044,23.76,37,0,1.62,MZ,2024-09-15
171,171,tsiombe,-25.3000,45.4833,20.94,75,0,1.04,MG,2024-09-15
201,201,jamestown,42.0970,-79.2353,24.80,43,0,3.09,US,2024-09-15
270,270,lupane,-18.9315,27.8070,21.07,25,0,2.95,ZW,2024-09-15
310,310,belmonte,-15.8631,-38.8828,22.99,84,0,3.50,BR,2024-09-15
316,316,mount isa,-20.7333,139.5000,21.87,30,0,4.12,AU,2024-09-15
326,326,vernon,50.2581,-119.2691,20.74,33,0,1.54,CA,2024-09-15
394,394,giado,31.9582,12.0203,21.27,68,0,4.22,LY,2024-09-15


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,65,
117,tostado,AR,-29.2320,-61.7692,28,
141,mutuali,MZ,-14.8706,37.0044,37,
171,tsiombe,MG,-25.3000,45.4833,75,
201,jamestown,US,42.0970,-79.2353,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":20,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
tostado - nearest hotel: Hotel Centro
mutuali - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
lupane - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
vernon - nearest hotel: Okanagan Royal Park Inn
giado - nearest hotel: فندق جادو السياحي
warner robins - nearest hotel: Comfort Inn & Suites - Near Robins Air Force Base Main Gate
ibiza - nearest hotel: Gran Hotel Montesol Ibiza, Curio Collection by Hilton
virginia - nearest hotel: Quirk Hotel
sandpoint - nearest hotel: K2 Inn
westport - nearest hotel: Norwalk Inn and Conference Center
nampula - nearest hotel: Hotel Lurio


,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,65,Hampton Inn Danbury
117,tostado,AR,-29.2320,-61.7692,28,Hotel Centro
141,mutuali,MZ,-14.8706,37.0044,37,No hotel found
171,tsiombe,MG,-25.3000,45.4833,75,No hotel found
201,jamestown,US,42.0970,-79.2353,43,DoubleTree Jamestown
270,lupane,ZW,-18.9315,27.8070,25,No hotel found
310,belmonte,BR,-15.8631,-38.8828,84,No hotel found
316,mount isa,AU,-20.7333,139.5000,30,Ibis Styles
326,vernon,CA,50.2581,-119.2691,33,Okanagan Royal Park Inn
394,giado,LY,31.9582,12.0203,68,فندق جادو السياحي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)